<a href="https://colab.research.google.com/github/stevehooker/welsh-dragons/blob/main/notebooks/comfyui_colab_fast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ComfyUI on Google Colab (FAST Edition)

**Version 3.3 - 31 Dec 2025**

**⚡ Target startup time: ~10-15 minutes** (down from 30-40 minutes)

## Strategy

1. **Symlink known large nodes** (hardcoded list - instant)
2. **Bulk copy everything else** (one rsync command - fast)
3. **Auto-detect dependencies** with cm-cli

**Setup Instructions:**
1. Enable GPU: Runtime → Change runtime type → T4 GPU
2. Run all cells: Runtime → Run all
3. Wait for the cloudflared URL

In [ ]:
#@title 0. Load Secrets (Optional)
import os

try:
    from google.colab import userdata
    secrets_found = []
    for key in ['ANTHROPIC_API_KEY', 'OPENAI_API_KEY', 'HF_TOKEN']:
        try:
            value = userdata.get(key)
            if value:
                os.environ[key] = value
                secrets_found.append(key)
        except:
            pass
    if secrets_found:
        print(f"✓ Loaded: {', '.join(secrets_found)}")
    else:
        print("ℹ️  No secrets found")
except Exception as e:
    print(f"ℹ️  Secrets not available")

In [ ]:
#@title 1. Mount Drive
import os

print("Mounting Google Drive...")
from google.colab import drive
drive.mount('/content/drive')

DRIVE_COMFYUI = "/content/drive/MyDrive/ComfyUI"
LOCAL_COMFYUI = "/content/ComfyUI"

if not os.path.exists(DRIVE_COMFYUI):
    raise RuntimeError(f"❌ ComfyUI not found at {DRIVE_COMFYUI}")

print(f"✓ Found ComfyUI on Drive")

In [ ]:
#@title 2. Copy to Local Storage (Fast Method)
#@markdown Uses rsync with hardcoded exclusions - NO slow size checks!

import os
import time

start_time = time.time()

# Known large custom nodes to EXCLUDE from copy (symlink instead)
# These contain model weights, not just code
LARGE_NODES = [
    'comfyui_controlnet_aux',  # ~3 GB - ControlNet preprocessor models
    'Bjornulf_custom_nodes',    # ~574 MB
    'RES4LYF',                  # ~473 MB
    'efficiency-nodes-comfyui', # ~459 MB
    'comfyui_queue_manager',    # ~236 MB
    'SeargeSDXL',               # ~210 MB
    'rgthree-comfy',            # ~183 MB
    'tbg-sam3',                 # ~143 MB
    'was-node-suite-comfyui',   # ~125 MB
    'comfyui-mixlab-nodes',     # ~103 MB
    'cg-use-everywhere',        # ~100 MB
]

# Clean previous
!rm -rf {LOCAL_COMFYUI}
!mkdir -p {LOCAL_COMFYUI}/custom_nodes

print("="*60)
print("Fast Copy using rsync (no slow size checks!)")
print("="*60)

# Build rsync exclude list
excludes = ' '.join([f"--exclude='custom_nodes/{n}'" for n in LARGE_NODES])
excludes += " --exclude='models' --exclude='input' --exclude='output'"

# Single rsync command - much faster than Python shutil
print("\n[1/3] Copying ComfyUI (excluding large nodes & models)...")
!rsync -a --info=progress2 {excludes} {DRIVE_COMFYUI}/ {LOCAL_COMFYUI}/

# Symlink the large custom nodes
print("\n[2/3] Symlinking large custom nodes...")
for node in LARGE_NODES:
    src = f"{DRIVE_COMFYUI}/custom_nodes/{node}"
    dst = f"{LOCAL_COMFYUI}/custom_nodes/{node}"
    if os.path.exists(src) and not os.path.exists(dst):
        os.symlink(src, dst)
        print(f"  🔗 {node}")

# Symlink model directories
print("\n[3/3] Symlinking models/input/output...")
for d in ['models', 'input', 'output']:
    src = f"{DRIVE_COMFYUI}/{d}"
    dst = f"{LOCAL_COMFYUI}/{d}"
    if os.path.exists(src) and not os.path.exists(dst):
        os.symlink(src, dst)
        print(f"  ✓ {d}")

elapsed = time.time() - start_time
print(f"\n✓ Setup complete in {elapsed:.1f} seconds")

In [ ]:
#@title 3. Install Dependencies
import time
start = time.time()

print("Installing dependencies...")
%cd {LOCAL_COMFYUI}

!pip install -q -r requirements.txt
!pip install -q --upgrade comfyui-frontend-package

print("\nScanning custom nodes for dependencies...")
!python custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies

!pip install -q --upgrade comfyui-frontend-package

print(f"\n✓ Dependencies installed in {time.time()-start:.1f}s")

In [ ]:
#@title 4. Verify GPU
import torch
print(f"PyTorch {torch.__version__}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print("✓ GPU ready!")
else:
    print("❌ No GPU! Runtime > Change runtime type > T4 GPU")

In [ ]:
#@title 5. Start ComfyUI
import subprocess, threading, time, socket, re

![ ! -f ~/cloudflared-linux-amd64.deb ] && wget -q -O ~/cloudflared-linux-amd64.deb https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i ~/cloudflared-linux-amd64.deb 2>/dev/null

cloudflared_url = None
start_time = time.time()

def start_tunnel(port=8188):
    global cloudflared_url
    for _ in range(120):
        try:
            s = socket.socket(); s.settimeout(1)
            if s.connect_ex(('127.0.0.1', port)) == 0: break
            s.close()
        except: pass
        time.sleep(0.5)
    
    proc = subprocess.Popen(["cloudflared", "tunnel", "--url", f"http://127.0.0.1:{port}"],
        stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    for line in proc.stderr:
        if "trycloudflare.com" in line:
            m = re.search(r'https://[a-z0-9-]+\.trycloudflare\.com', line)
            if m: cloudflared_url = m.group(0); break

threading.Thread(target=start_tunnel, daemon=True).start()

%cd {LOCAL_COMFYUI}
print("Starting ComfyUI...\n")

proc = subprocess.Popen(["python", "main.py", "--dont-print-server"],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1)

ready = False
for line in proc.stdout:
    print(line, end='')
    if "Import times for custom nodes:" in line and not ready:
        ready = True; time.sleep(2)
        elapsed = time.time() - start_time
        for _ in range(20):
            if cloudflared_url:
                print(f"\n{'='*60}")
                print(f"🎉 READY in {elapsed:.0f}s: {cloudflared_url}")
                print(f"{'='*60}\n")
                break
            time.sleep(1)

## Notes

**Why v3.2 was slow:** The `du -sm` size check had to traverse every file over Google Drive's slow FUSE mount. Each folder check took ~10 minutes!

**v3.3 fix:** Hardcoded list of known large nodes + single rsync command. No per-folder checks.

**To add more exclusions:** Edit the `LARGE_NODES` list in Section 2.